# Classification Model Evaluation

In [22]:
#start with all the import statments
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import os
import acquire
import prepare

import warnings
warnings.filterwarnings("ignore")

from scipy import stats
from sklearn.model_selection import train_test_split
from env import host, user, password

from sklearn.tree import DecisionTreeClassifier

The Confusion Matrix

 <img src="https://miro.medium.com/max/1334/1*3yGLac6F4mTENnj5dBNvNQ.jpeg" alt="Confusion Matrix">

Now we will get into the weeds of this project, I have barely enough finished to get going but that's what Im ma have to go with
if I want any sleep that is

<b>
The spirit of this journal is to review the ciriculmn and be able to do what is asked of me.
    <hr>
    <i>MVP, Vasiliy</i>
</b>

In [8]:
df = acquire.telco_churn()
df.head(5)

,payment_type_id,internet_service_type_id,contract_type_id,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,...,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type
0,2,1,2,0002-ORFBO,Female,0,Yes,Yes,9,Yes,...,Yes,Yes,No,Yes,65.6,593.3,No,One year,DSL,Mailed check
1,2,1,1,0003-MKNFE,Male,0,No,No,9,Yes,...,No,No,Yes,No,59.9,542.4,No,Month-to-month,DSL,Mailed check
2,1,2,1,0004-TLHLJ,Male,0,No,No,4,Yes,...,No,No,No,Yes,73.9,280.85,Yes,Month-to-month,Fiber optic,Electronic check
3,1,2,1,0011-IGKFF,Male,1,Yes,No,13,Yes,...,No,Yes,Yes,Yes,98.0,1237.85,Yes,Month-to-month,Fiber optic,Electronic check
4,2,2,1,0013-EXCHZ,Female,1,Yes,No,3,Yes,...,Yes,Yes,No,Yes,83.9,267.4,Yes,Month-to-month,Fiber optic,Mailed check


now that I have my features to work through let me find the statistical value of all of them
<hr>

In [9]:
train, validate, test = prepare.prep_telco_data(df, target='churn')
train.shape, validate.shape, test.shape

((3937, 47), (1688, 47), (1407, 47))

In [10]:
alpha = 0.05

In [11]:
senior_citizen_ctab = pd.crosstab(train.senior_citizen, train.churn)
senior_citizen_ctab

churn,No,Yes
senior_citizen,,
0,2527,775
1,364,271


In [12]:
chi2, p_senior, degf, expected_senior = stats.chi2_contingency(senior_citizen_ctab)
print(expected_senior)
print(f"p-val: {p_senior}")

[[2424.70967742  877.29032258]
 [ 466.29032258  168.70967742]]
p-val: 1.7561315684822695e-23


In [14]:
if p_senior < alpha:
    print("We reject the null hypothesis.")
else:
    print("We fail to reject the null hypothesis.")

We reject the null hypothesis.


In [19]:
dsl_ctab = pd.crosstab(train.internet_service_type, train.churn)
dsl_ctab

churn,No,Yes
internet_service_type,,
DSL,1101,247
Fiber optic,987,735
None,803,64


In [20]:
#we can only do confusion matrix in a 2 x 2 format so I have to transform my data into a form that can do that
#maybe having an encoded version of the variables i want
#at present I do not have the luxury of time 

In [27]:
def feature_target_selector(features, target, train=train, validate=validate, test=test):
    X_train = train[features]
    y_train = train[target]
    X_validate = validate[features]
    y_validate = validate[target]
    X_test = test[features]
    y_test = test[target]
    return X_train, y_train, X_validate, y_validate, X_test, 
train.columns

Index(['gender', 'senior_citizen', 'partner', 'dependents', 'tenure',
       'phone_service', 'multiple_lines', 'online_security', 'online_backup',
       'device_protection', 'tech_support', 'streaming_tv', 'streaming_movies',
       'paperless_billing', 'monthly_charges', 'total_charges', 'churn',
       'contract_type', 'internet_service_type', 'payment_type',
       'gender_encoded', 'partner_encoded', 'dependents_encoded',
       'phone_service_encoded', 'paperless_billing_encoded', 'churn_encoded',
       'multiple_lines_No phone service', 'multiple_lines_Yes',
       'online_security_No internet service', 'online_security_Yes',
       'online_backup_No internet service', 'online_backup_Yes',
       'device_protection_No internet service', 'device_protection_Yes',
       'tech_support_No internet service', 'tech_support_Yes',
       'streaming_tv_No internet service', 'streaming_tv_Yes',
       'streaming_movies_No internet service', 'streaming_movies_Yes',
       'contract_type_

In [28]:
scores = []
for i in range(1, 41):
    tree = DecisionTreeClassifier(random_state = 1, max_depth = i)
    tree.fit(X_train, y_train)
    scores.append(tree.score(X_train, y_train))
max_train_accuracy = max(scores)
max_train_index = scores.index(max_train_accuracy)
optimal_train_depth = max_train_index + 1
print(f'The optimal max_depth is {optimal_train_depth}')
print(f'The accuracy on the train data at this depth is {round(max_train_accuracy, 3)}')

plt.plot(range(1,41), scores)
plt.show()

NameError: name 'X_train' is not defined

In [29]:
features = ['month_to_month_encoded', 'tenure', 'fiber_optic_encoded', 'electronic_check_encoded', 'has_internet', 'monthly_charges']
target = ['churn_encoded']
X_train, y_train, X_validate, y_validate, X_test, y_test = feature_target_selector(features, target)

KeyError: "['month_to_month_encoded', 'fiber_optic_encoded', 'electronic_check_encoded', 'has_internet'] not in index"

In [30]:
df.head()

,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type
0,Female,0,Yes,Yes,9,Yes,No,No,Yes,No,Yes,Yes,No,Yes,65.6,593.3,No,One year,DSL,Mailed check
1,Male,0,No,No,9,Yes,Yes,No,No,No,No,No,Yes,No,59.9,542.4,No,Month-to-month,DSL,Mailed check
2,Male,0,No,No,4,Yes,No,No,No,Yes,No,No,No,Yes,73.9,280.85,Yes,Month-to-month,Fiber optic,Electronic check
3,Male,1,Yes,No,13,Yes,No,No,Yes,Yes,No,Yes,Yes,Yes,98.0,1237.85,Yes,Month-to-month,Fiber optic,Electronic check
4,Female,1,Yes,No,3,Yes,No,No,No,No,Yes,Yes,No,Yes,83.9,267.4,Yes,Month-to-month,Fiber optic,Mailed check


In [31]:
# so from here I have 6 hours until this day is over and i would like to have this present ready by tonight 
# ideally 10 pm giving me enough time for sleep and script practice 

maybe i can pour my energy into doing markdown to edit together my presentation
<br>
<p>
it is okay to skip steps to just get the thing done, sometimes its all you need
like in construction, just get the damn thing finished
they don't care how they don't care why
</p>

alright so with me following someone else code it looks like I am going to have to build my own version
<hr>
wait, I can also do is build out my prepare file to be more robust
look buddy either we do this and learn something or learn nothing at all

now i am in the process of finding what i have and don't have then making the last sprint to the finish